In [4]:
import os

# Print the current working directory
print(os.getcwd())


/mnt/speech6/bagher/aentezari/project


In [3]:
import os

# drive_path = '/content/drive/MyDrive'  # Adjust path if using google drive
drive_path = os.getcwd()  # Adjust path if using google drive
if os.path.exists(drive_path):
  folder_names = [f for f in os.listdir(drive_path) if os.path.isdir(os.path.join(drive_path, f))]
  print(folder_names)
else:
  print("Drive path does not exist.")

['.venv', '.git', 'dataset', 'src']


In [9]:
import torch

import random

import numpy as np

import pandas as pd

import mne

In [1]:
import os

def get_directory_size(directory):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for file in filenames:
            file_path = os.path.join(dirpath, file)
            # Skip if the file is a symbolic link
            if not os.path.islink(file_path):
                total_size += os.path.getsize(file_path)
    return total_size

# Example usage
directory_path = 'dataset/derivatives/embeddings'
size_in_bytes = get_directory_size(directory_path)
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"Size of '{directory_path}' is {size_in_bytes} bytes ({size_in_mb:.2f} MB)")

Size of 'dataset/derivatives/embeddings' is 1063393452 bytes (1014.13 MB)


In [ ]:
# Comment this code if you have the preprocessed dataset
from scipy.signal import resample

dataset_dir = 'dataset/'
rootdir = dataset_dir + 'derivatives'
participants_file = dataset_dir + 'participants.tsv'
output_dir = dataset_dir + 'derivatives/processed_dataset'


chunk_size = 15000  # 30 seconds of data at 500 Hz
overlap = 7500   # 15 seconds overlap (7500 samples)
new_sampling_rate = 100  # Target sampling rate
original_sampling_rate = 500  # Original sampling rate

# Load participants information
participants_df = pd.read_csv(participants_file, sep='\t')

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process each subject
subjects = os.listdir(rootdir)
for subject in subjects:
    subject_dir = os.path.join(rootdir, subject)

    if not os.path.isdir(subject_dir):
        continue

    subject_dir += '/eeg'

    # Match participant_id in participants.tsv
    participant_row = participants_df[participants_df['participant_id'] == subject]
    if participant_row.empty:
        print(f"Warning: No matching entry in participants.tsv for {subject}")
        continue

    # Extract labels
    labels = {
        "MMSE": int(participant_row["MMSE"]),
        "Group": participant_row["Group"].values[0],
        "Age": int(participant_row["Age"]),
        "Gender": participant_row["Gender"].values[0]
    }

    subject_output_dir = os.path.join(output_dir, subject)
    if not os.path.exists(subject_output_dir):
        os.makedirs(subject_output_dir)

    # Process each file in the subject's directory
    for file_name in os.listdir(subject_dir):
        if file_name.endswith('.set'):
            file_path = os.path.join(subject_dir, file_name)

            try:
                # Load EEG data
                raw = mne.io.read_raw_eeglab(file_path, preload=True)
                data = raw.get_data()  # Get the raw EEG data as a numpy array
            except Exception as e:
                print(f"Failed to process {file_path}: {e}")
                continue

            # Split data into 30-second chunks with 15-second overlap
            num_samples = data.shape[1]
            start = 0
            chunk_index = 0
            while start + chunk_size <= num_samples:
                end = start + chunk_size
                chunk_data = data[:, start:end]
                # Downsample the chunk
                resampled_chunk_data = resample(
                    chunk_data,
                    int(chunk_data.shape[-1] * new_sampling_rate / original_sampling_rate),
                    axis=-1
                )
                # Create sample dictionary
                sample = {
                    "data": resampled_chunk_data,
                    "label": labels['Group']
                }

                # Save chunk
                output_file_name = file_name.replace('.set', f'_chunk_{chunk_index:03d}.pt')
                output_file_path = os.path.join(subject_output_dir, output_file_name)
                print(f'subject: {file_name} sample :{chunk_index}:start: {start} end: {end} start + chunk_size: {start + chunk_size }\n Path: {output_file_path}')
                torch.save(sample, output_file_path)
                print(resampled_chunk_data.shape)

                # Update indices
                start += overlap
                chunk_index += 1
            # Handle the remaining data at the end
            # if start < num_samples:
            #     chunk_data = data[:, start:num_samples]

            #     resampled_chunk_data = resample(
            #         chunk_data,
            #         int(chunk_data.shape[-1] * new_sampling_rate / original_sampling_rate),
            #         axis=-1
            #     )

            #     sample = {
            #         "data": resampled_chunk_data,
            #         "label": labels['Group']
            #     }

            #     output_file_name = file_name.replace('.set', f'_chunk_{chunk_index:03d}.pt')
            #     output_file_path = os.path.join(subject_output_dir, output_file_name)
            #     torch.save(sample, output_file_path)

            #     print(f'Subject: {file_name}, Final Sample: {chunk_index}, Start: {start}, End: {num_samples}, Saved at: {output_file_path}')

print("Processing complete. Processed data saved in:", output_dir)

In [8]:
# data = torch.load('/content/drive/MyDrive/Colab Notebooks/ds004504/derivatives/sub-005/eeg/sub-005_task-eyesclosed_eeg.set')
file_size = os.path.getsize('dataset/derivatives/sub-001/eeg/sub-001_task-eyesclosed_eeg.set')
file_size_mb = file_size / (1024 ** 2)
print(f"File .set size: {file_size_mb:.2f} MB")

raw = mne.io.read_raw_eeglab('dataset/derivatives/sub-001/eeg/sub-001_task-eyesclosed_eeg.set', preload=True)
data = raw.get_data()
print(data.shape)
torch.save(data, 'test.pt')

file_size = os.path.getsize('test.pt')
file_size_mb = file_size / (1024 ** 2)
print(f"File .pt size: {file_size_mb:.2f} MB")
os.remove('test.pt')

File .set size: 23.23 MB
(19, 299900)
File .pt size: 66.02 MB
